In [ ]:
### !git clone https://github.com/your-username/vascular-llm-qa.git
!git clone https://github.com/sashajh2/vascular-llm-qa.git

Cloning into 'vascular-llm-qa'...
remote: Enumerating objects: 11522, done.
remote: Counting objects: 100% (3287/3287), done.
remote: Compressing objects: 100% (3211/3211), done.
remote: Total 11522 (delta 86), reused 3257 (delta 69), pack-reused 8235 (from 1)
Receiving objects: 100% (11522/11522), 79.33 MiB | 9.08 MiB/s, done.
Resolving deltas: 100% (1738/1738), done.
Updating files: 100% (3169/3169), done.


In [1]:
import pickle
import pandas as pd
import numpy as np
import pickle
import csv
import os
import time

In [2]:
from google.colab import drive, auth
from transformers import AutoModelForCausalLM, AutoTokenizer
print(auth.authenticate_user(project_id='mit-mlhc'))

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

None
Mounted at /content/drive


In [3]:
id_question = {}
with open('question_context.pickle', "rb") as f:
  id_question = pickle.load(f)


In [4]:
worksheet = gc.open('My cool spreadsheet').sheet1
rows = worksheet.get_all_values()
results = pd.DataFrame.from_records(rows)
results.columns = results.iloc[0]  # Set the first row as column names
results.drop(results.index[0], inplace=True)  # Drop the first row since it's now the header
results

IndexError: single positional indexer is out-of-bounds

In [5]:
p_response = {} # (prompt id, question id, model) -> model raw response
for idx, row in results.iterrows():
  key = row["prompt_id"], row["question_id"], row["model"]
  p_response[key] = row["response"]
print(len(p_response))

0


In [6]:
pre_prompts = {
1: "",
2: """You are a vascular surgeon responsible for teaching medical students. Provide concise, medically accurate, and ethically appropriate answers based on current standards of care. Focus only on clinical education relevant to surgery.""",
3: """You are a helpful medical knowledge assistant specializing in teaching vascular surgery to medical students. Provide useful, complete, and scientifically-grounded answers to clinical questions relevant to surgery and medical care.""",
4: """Answer the following multiple choice question from the medical domain based on these instructions. Choose one or more of the provided answer choices which are correct and output them under the heading "Answer".""",
5: """You are a helpful medical knowledge assistant specializing in teaching vascular surgery to medical students.
Answer the following multiple choice question from the medical domain based on the following instructions.
1. Select the one or more correct options and provide the option(s) under the heading "Answer".
2. Always select one or more of the provided options as the answer.
3. If the options are ambiguous or the question does not have enough context, select the ones that best answer the question.
4. If the question is not clinically relevant to vascular surgery, please respond with, "I'm sorry, I cannot answer this question." """
}

few_shot = """Question:
Which of the following is the most common site for peripheral arterial disease (PAD)?
A. Carotid artery
B. Femoral-popliteal artery
C. Renal artery
D. Subclavian artery
Answer:
B. Femoral-popliteal artery

Question:
Which of the following are common risk factors for developing an abdominal aortic aneurysm (AAA)?
A. Smoking
B. Male gender
C. Hypertension
D. Type 1 diabetes mellitus
E. Family history of AAA
Answer:
A. Smoking
B. Male gender
C. Hypertension
E. Family history of AAA
"""

pre_prompts[6] = few_shot

In [7]:
def format_mcq_example(preprompt, example):
    letters = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    choices = example["choices"]
    lettered_choices = [f"{letters[i]}. {choice}" for i, choice in enumerate(choices)]
    # (Instructions) - (context) - question - options
    prompt = f"Question: {example['question']}\nOptions:\n" + "\n".join(lettered_choices) + "\nAnswer:"
    context = example.get("context", [])
    if context:
        context_text = "\n".join([f"Document {i+1}: {chunk}" for i, chunk in enumerate(context)])
        prompt = f'Context:\n{context_text}{prompt}'

    explanation = example.get("explanation", "[explanation]")
    source = example.get("source", "[source]")
    if preprompt:
      prompt = f"Instructions:\n{preprompt}\n{prompt}"


    target = f"{', '.join(example['correct_answer'])} - {explanation} - {source}"
    return prompt, target





prompts = []
for prompt_id, preprompt in pre_prompts.items():
  if prompt_id != 1:
    continue
  for question in id_question.values():
    p = {"query": {"role": "user", "content": format_mcq_example(preprompt, question)[0]},
         "prompt_id": prompt_id,
         "question_id": question["question_id"]}
    prompts.append(p)

import random
random.shuffle(prompts)
print(len(prompts))

451


In [ ]:
models = ["FreedomIntelligence/Apollo-0.5B", "johnsnowlabs/JSL-MedPhi2-2.7B", "skumar9/Llama-medx_v3.2", "microsoft/Phi-4-mini-instruct"]

In [ ]:
import transformers
from transformers import AutoTokenizer
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
import torch

In [ ]:
from tqdm import tqdm
#p_response = {} # (prompt id, question id, model) -> model raw response

def run_prompts(prompts, id_question, pipe, model_name):
  for prompt in tqdm(prompts):
    key = (prompt["prompt_id"], prompt["question_id"], model_name)
    if key in p_response:
      continue # already recorded

    messages = [prompt["query"]]
    raw_response = pipe(messages, max_new_tokens=256, do_sample=True, temperature=0.1, top_k=50, top_p=0.95)
    p_response[key] = raw_response

    question = id_question[prompt["question_id"]]
    response = raw_response[0]['generated_text'][1]['content']
    answer = question['full_answers']
    entry = [*key, "\n ".join(answer), response, ""]
    worksheet.append_row(entry)
    print(key, answer, response)

In [ ]:
# Load tokenizer and pipeline for phi
ap_tokenizer = AutoTokenizer.from_pretrained(models[3])

ap_pipeline = transformers.pipeline(
    "text-generation",
    model=models[3],
    tokenizer=ap_tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.93k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.91M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/15.5M [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.77G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
run_prompts(prompts, id_question, ap_pipeline, models[3])

  0%|          | 1/451 [00:09<1:12:28,  9.66s/it]

(1, '56.1', 'microsoft/Phi-4-mini-instruct') ['D. Anticoagulation'] B. Resuscitation

In the context of septic shock, the immediate priority is to stabilize the patient. This typically involves resuscitation, which includes securing the airway, ensuring adequate breathing and oxygenation, and initiating fluid resuscitation to maintain blood pressure and organ perfusion. Once the patient is stabilized, further steps such as obtaining blood cultures before antibiotic administration, initiating broad-spectrum antibiotics, and investigating the cause of sepsis can be taken. Anticoagulation is not a standard initial step in managing septic shock.


  0%|          | 2/451 [00:15<55:46,  7.45s/it]  

(1, '56.5', 'microsoft/Phi-4-mini-instruct') ['C. Acute pulmonary edema'] B. Acute pulmonary thromboembolism

Massive edema, particularly when associated with venous insufficiency or deep vein thrombosis (DVT), can lead to the formation of blood clots. These clots can dislodge and travel to the lungs, causing a potentially life-threatening condition known as pulmonary embolism (PE). This is considered one of the most serious risks associated with the management of massive edema.


  1%|          | 3/451 [00:22<52:40,  7.05s/it]

...

(1, '48', 'microsoft/Phi-4-mini-instruct') ['C. has the suprapleural membrane attached to its inner margin, behind the subclavian groove', 'D. is the shortest, widest, flattest and most curved rib'] B. has a slip of serratus anterior and serratus posterior superior attached
C. has the suprapleural membrane attached to its inner margin, behind the subclavian groove

The first rib is unique among the ribs in several ways. It has a slip of the serratus anterior and serratus posterior superior muscles attached to it, which helps in the movement of the scapula. Additionally, the suprapleural membrane, also known as the chyle sac, is attached to the inner margin of the first rib, behind the subclavian groove. This membrane helps to protect the thoracic duct as it passes through the thoracic cavity.

Option A is incorrect because the subclavian groove on the first rib contains the subclavian artery and vein, not just the artery. Option D is incorrect because the first rib is not the shortest,

100%|█████████▉| 450/451 [1:10:07<00:10, 10.08s/it]

(1, '401', 'microsoft/Phi-4-mini-instruct') ['A. Full leaded face shield, lead apron, and personal radiation badge'] A. Full leaded face shield, lead apron, and personal radiation badge

Discussion: The most appropriate PPE includes a full leaded face shield, lead apron, and radiation badge making answer A the correct answer. This will provide the most significant protection to the head and neck as well as vital organs of the body during the procedure. The radiation badge will help with long-term protection as it allows for cumulative tracking of exposure. A lead apron alone and or lead glasses alone is inadequate for optimal protection making answers B, C, and D incorrect. A full-face shield will be of greater protection than glasses which makes answer E incorrect.


100%|██████████| 451/451 [1:10:12<00:00,  9.34s/it]

(1, '24.5', 'microsoft/Phi-4-mini-instruct') ['A. The medial gastrocnemius muscle is involved in almost 80% of cases of PAE.', 'C. Venous entrapment is concomitant in 20% of cases of PAE.', 'D. More than one structure may be the cause of arterial entrapment.'] Based on the provided information, the correct answers are:

D. More than one structure may be the cause of arterial entrapment.
E. Classification of arterial entrapment includes 12 different types.

The other options (A, B, and C) are not supported by the provided information.


In [46]:
!pip install google-cloud-sdk

ERROR: Could not find a version that satisfies the requirement google-cloud-sdk (from versions: none)
ERROR: No matching distribution found for google-cloud-sdk


In [47]:
!gcloud config set project macro-gadget-458902-i2

Updated property [core/project].


In [48]:
!gcloud config list project

[core]
project = macro-gadget-458902-i2

Your active configuration is: [default]


In [49]:
from google.cloud import storage

# Create a client object
client = storage.Client()

In [50]:
!gcloud beta billing projects describe macro-gadget-458902-i2

billingAccountName: billingAccounts/010968-A1D9E2-69C8E5
billingEnabled: true
name: projects/macro-gadget-458902-i2/billingInfo
projectId: macro-gadget-458902-i2


In [51]:
#!gcloud auth login
!gcloud config set project macro-gadget-458902-i2

Updated property [core/project].


In [52]:
bucket = client.bucket('mlhcbucket')

In [53]:
bucket.exists()

True

In [54]:
blobs = bucket.list_blobs()
for blob in blobs:
  print(blob.name)

In [55]:
print(blobs)

In [56]:
blob = bucket.blob('gemini_batch_requests.jsonl')

In [58]:
blob

<Blob: mlhcbucket, gemini_batch_requests.jsonl, None>

In [8]:
# run RAG examples on gemini

# Set up Google Vertex API call
# NOTE -- can comment this out
# (not needed for hugging face models)
import vertexai
from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    Image,
    Part,
    SafetySetting,
)
from vertexai.batch_prediction import BatchPredictionJob
import json
from google.cloud import storage

# replace with project ID from Google Cloud Platform
PROJECT_ID = "macro-gadget-458902-i2"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [23]:
def df_to_jsonl_gcs(prompts, bucket_name, blob_name):
    """Converts a DataFrame to JSONL and uploads to Google Cloud Storage.

    Args:
        df: Pandas DataFrame with a 'text' column.
        bucket_name: Name of your Google Cloud Storage bucket.
        blob_name: Desired name for the JSONL file on GCS.
    """



    # Write JSONL data to a string buffer
    jsonl_data = ""
    for prompt in prompts:
        text = prompt["query"]["content"]
        json_data = {
            "prompt_id": prompt["prompt_id"],
            "model": "gemini-2.0-flash-001",
            "question_id": prompt["question_id"],
            "request": {
                "contents": [
                    {
                        "role": "user",
                        "parts": [{"text": text}]
                    }
                ],
                "generationConfig": {"temperature": 0.1, "maxOutputTokens": 256},

            }
        }
        jsonl_data += json.dumps(json_data) + '\n'
    return jsonl_data

    # Upload the JSONL data to GCS
    blob.upload_from_string(jsonl_data, content_type='application/jsonl')
    print(f"JSONL file uploaded to gs://{bucket_name}/{blob_name}")

    return f"gs://{bucket_name}/{blob_name}"


    # Specify a custom bucket name
BUCKET_NAME = 'mlhcbucket'

data = df_to_jsonl_gcs(prompts, BUCKET_NAME, 'gemini_batch_requests.jsonl')


In [28]:
blob.upload_from_string(data, content_type='application/jsonl')


Forbidden: 403 POST https://storage.googleapis.com/upload/storage/v1/b/mlhcbucket/o?uploadType=multipart: {
  "error": {
    "code": 403,
    "message": "The billing account for the owning project is disabled in state closed",
    "errors": [
      {
        "message": "The billing account for the owning project is disabled in state closed",
        "domain": "global",
        "reason": "accountDisabled",
        "locationType": "header",
        "location": "Authorization"
      }
    ]
  }
}
: ('Request failed with status code', 403, 'Expected one of', <HTTPStatus.OK: 200>)

In [59]:
import json

def df_to_jsonl_file(prompts, filename):
    """Converts a list of prompts to JSONL and writes to a local file.

    Args:
        prompts: A list of prompts.
        filename: The name of the file to write the JSONL data to.
    """

    with open(filename, 'w') as f:
        for prompt in prompts:
            text = prompt["query"]["content"]
            json_data = {
                "prompt_id": prompt["prompt_id"],
                "model": "gemini-2.0-flash-001",
                "question_id": prompt["question_id"],
                "request": {
                    "contents": [
                        {
                            "role": "user",
                            "parts": [{"text": text}]
                        }
                    ],
                    "generationConfig": {"temperature": 0.1, "maxOutputTokens": 256},

                }
            }
            f.write(json.dumps(json_data) + '\n')

    print(f"JSONL data written to {filename}")


# Example usage
filename = 'gemini_batch_requests.jsonl'  # Choose your desired filename
df_to_jsonl_file(prompts, filename)  # Assuming 'prompts' is your list of prompts

JSONL data written to gemini_batch_requests.jsonl


In [62]:
input_uri = 'gs://mlhcbucket/gemini_batch_requests.jsonl'
output_uri = f"gs://{BUCKET_NAME}/batch-prediction/"

# Submit a batch prediction job with Gemini model
batch_prediction_job = BatchPredictionJob.submit(
    source_model="gemini-2.0-flash-001",
    #source_model="gemini-1.5-flash-001",
    #source_model="jsl-medphi2-2.7b",
    #"projects/552077056809/locations/us-central1/endpoints/1130919177423421440",
    input_dataset=input_uri,
    output_uri_prefix=output_uri,
)

# Check job status
print(f"Job resource name: {batch_prediction_job.resource_name}")
print(f"Model resource name with the job: {batch_prediction_job.model_name}")
print(f"Job state: {batch_prediction_job.state.name}")

# Refresh the job until complete
while not batch_prediction_job.has_ended:
    time.sleep(5)
    batch_prediction_job.refresh()

# Check if the job succeeds
if batch_prediction_job.has_succeeded:
    print("Job succeeded!")
else:
    print(f"Job failed: {batch_prediction_job.error}")

# Check the location of the output
print(f"Job output location: {batch_prediction_job.output_location}")

# # Example response:
# #  Job output location: gs://your-bucket/gen-ai-batch-prediction/prediction-model-year-month-day-hour:minute:second.12345


# # NOTE:
# # After job finishes running, copied data over to my drive and loaded it
# # into a pandas dataframe
# # Job took 2 min 55 sec

INFO:vertexai.batch_prediction._batch_prediction:BatchPredictionJob created. Resource name: projects/201055451681/locations/us-central1/batchPredictionJobs/4229961024649822208
INFO:vertexai.batch_prediction._batch_prediction:To use this BatchPredictionJob in another session:
INFO:vertexai.batch_prediction._batch_prediction:job = batch_prediction.BatchPredictionJob('projects/201055451681/locations/us-central1/batchPredictionJobs/4229961024649822208')
INFO:vertexai.batch_prediction._batch_prediction:View Batch Prediction Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/4229961024649822208?project=201055451681


Job resource name: projects/201055451681/locations/us-central1/batchPredictionJobs/4229961024649822208
Model resource name with the job: publishers/google/models/gemini-2.0-flash-001
Job state: JOB_STATE_PENDING
Job succeeded!
Job output location: gs://mlhcbucket/batch-prediction/prediction-model-2025-05-05T03:28:55.083682Z


In [64]:
# NOT NEEDED FOR HUGGING FACE MODELS
# load predictions from vertex
#predictions_fname = '/content/drive/MyDrive/llm_eval_vertex_response.jsonl'
predictions_fname = 'gemini_post_rag.jsonl'
predictions_df = pd.read_json(path_or_buf=predictions_fname, lines=True)


In [66]:
# NOT NEEDED FOR HUGGING FACE
def extract_text(response):
  """Extracts text from the response dictionary, handling potential KeyError."""
  try:
    return response['candidates'][0]['content']['parts'][0]['text']
  except (KeyError, IndexError, TypeError):
    # Handle cases where 'parts' key is missing or empty
    print(response)
   # Or any other appropriate default value
    return ''

predictions_df['summary'] = predictions_df['response'].apply(extract_text)
predictions_df

,question_id,model,prompt_id,status,processed_time,request,response,summary
0,419,gemini-2.0-flash-001,1,,2025-05-05 03:32:08.720000+00:00,{'contents': [{'parts': [{'text': 'Context:\nD...,{'candidates': [{'avgLogprobs': -0.16134893379...,The correct answer is **A. Stanford A dissecti...
1,32.1,gemini-2.0-flash-001,1,,2025-05-05 03:32:08.727000+00:00,{'contents': [{'parts': [{'text': 'Context:\nD...,{'candidates': [{'avgLogprobs': -0.17606310376...,"Okay, let's analyze the provided documents and..."
2,41.6,gemini-2.0-flash-001,1,,2025-05-05 03:32:08.728000+00:00,{'contents': [{'parts': [{'text': 'Context:\nD...,{'candidates': [{'avgLogprobs': -0.17053280998...,"Based on the provided context, here's a breakd..."
3,401,gemini-2.0-flash-001,1,,2025-05-05 03:32:08.728000+00:00,{'contents': [{'parts': [{'text': 'Context:\nD...,{'candidates': [{'avgLogprobs': -0.07246243127...,"The answer is **A. Full leaded face shield, le..."
4,52.5,gemini-2.0-flash-001,1,,2025-05-05 03:32:08.733000+00:00,{'contents': [{'parts': [{'text': 'Context:\nD...,{'candidates': [{'avgLogprobs': -0.16047630310...,Based on the provided documents and the patien...
...,...,...,...,...,...,...,...,...
446,40.4,gemini-2.0-flash-001,1,,2025-05-05 03:32:25.581000+00:00,{'contents': [{'parts': [{'text': 'Context:\nD...,{'candidates': [{'avgLogprobs': -0.13143135519...,"Based on the provided documents, here's an ana..."
447,57.2,gemini-2.0-flash-001,1,,2025-05-05 03:32:08.722000+00:00,{'contents': [{'parts': [{'text': 'Context:\nD...,{'candidates': [{'avgLogprobs': -0.12899301147...,"Based on the provided documents, here's an ana..."
448,57.3,gemini-2.0-flash-001,1,,2025-05-05 03:32:12.325000+00:00,{'contents': [{'parts': [{'text': 'Context:\nD...,{'candidates': [{'avgLogprobs': -0.15896653559...,"Based on the provided documents, here's an ana..."
449,373,gemini-2.0-flash-001,1,,2025-05-05 03:32:15.516000+00:00,{'contents': [{'parts': [{'text': 'Context:\nD...,{'candidates': [{'avgLogprobs': -0.18681633472...,"Based on the provided documents, the correct a..."


In [67]:
appended = []
# NOT NEEDED FOR HUGGING FACE
# write response back to spreadsheet
# vertex_df.drop(columns='id', inplace=True)
# vertex_df

#worksheet = gc.open('test_sheet').sheet1
#cols = predictions_df.columns.tolist()
#worksheet.append_row(["prompt_id", "question_id", "model", "answer", "response", "correct"])
print(len(appended))
for index, row in predictions_df.iterrows():
  question = id_question[row["question_id"]]
  answer = "\n".join(question['full_answers'])
  r = [row["prompt_id"], row["question_id"], row["model"], answer, row["summary"], ""]
  if r in appended:
    continue
  print(r)
  #worksheet.append_row(r)
  appended.append(r)
# dumped it to a different sheet, then copy and pasted it manually to avoid overwriting the actual spreadsheet
# manually evaluated responses


0
[1, '419', 'gemini-2.0-flash-001', 'A. Stanford A dissection.\nC. de Bakey I dissection.', "The correct answer is **A. Stanford A dissection.**\n\nHere's why:\n\n*   **Stanford Classification:** The Stanford classification system divides aortic dissections into two types:\n    *   **Type A:** Involves the ascending aorta (regardless of the site of origin or extent).\n    *   **Type B:** Does not involve the ascending aorta.\n*   **de Bakey Classification:** The de Bakey classification system divides aortic dissections into three types:\n    *   **Type I:** Originates in the ascending aorta and extends at least to the aortic arch, possibly further down the descending aorta.\n    *   **Type II:** Originates in and is confined to the ascending aorta.\n    *   **Type III:** Originates in the descending aorta and extends distally (Type IIIa) or both proximally and distally (Type IIIb).\n\nIn this case, the dissection involves the ascending aorta, aortic arch, and descending aorta. Since t

In [68]:
resdf = pd.DataFrame(appended, columns=["prompt_id", "question_id", "model", "answer", "response", "correct"])
resdf
resdf.to_csv('vertex_post_eval.csv')